In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf


In [ ]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth',None)
pd.reset_option('display.max_colwidth')

### Number of questions

In [ ]:
data_df=pd.read_csv('final_data_project.csv')

In [ ]:
data_df['MinScore'] = pd.to_numeric(data_df['MinScore'], errors='coerce')
data_df['MedScore'] = pd.to_numeric(data_df['MedScore'], errors='coerce')
data_df['MaxScore'] = pd.to_numeric(data_df['MaxScore'], errors='coerce')

In [ ]:
rows = []

# Process each answer column and assign the appropriate score label
for idx, row in data_df.iterrows():
    rows.append({
        'UserReputation': row['UserReputation_Min'],
        'AnswerTime': row['AnswerTime_Min'],
        'Score': 0
    })

    rows.append({
        'UserReputation': row['UserReputation_Med'],
        'AnswerTime': row['AnswerTime_Med'],
        'Score': 1
    })

    rows.append({
        'UserReputation': row['UserReputation_Max'],
        'AnswerTime': row['AnswerTime_Max'],
        'Score': 2
    })

transformed_data = pd.DataFrame(rows)

transformed_data['UserReputation'] = pd.to_numeric(transformed_data['UserReputation'], errors='coerce')
transformed_data['AnswerTime'] = pd.to_numeric(transformed_data['AnswerTime'], errors='coerce')
transformed_data['Score'] = transformed_data['Score'].astype(int)

data_df = transformed_data

In [ ]:
data_df.shape

(69552, 3)

In [ ]:
# Define the custom loss function with updated penalties
def custom_loss(y_true, y_pred):
    # Convert y_true back to class labels
    y_true_classes = tf.argmax(y_true, axis=1)
    y_pred_classes = tf.argmax(y_pred, axis=1)

    penalty_matrix = tf.constant([[0, 1.50, 2.0],  # True class 0
                                  [1.50, 0, 1.50],  # True class 1
                                  [2.0, 1.50, 0]], dtype=tf.float32)  # True class 2

    penalties = tf.gather(penalty_matrix, y_true_classes)

    base_loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred)

    custom_loss = base_loss * tf.gather_nd(penalties, tf.stack([y_true_classes, y_pred_classes], axis=1))

    return tf.reduce_mean(custom_loss)


In [ ]:
# Separate the features and the target variable
X = data_df[['UserReputation', 'AnswerTime']]
y = data_df['Score']

# One-hot encode the target variable
y_encoded = to_categorical(y, num_classes=3)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = Sequential()
model.add(Dense(64, input_dim=2, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss=custom_loss, metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test),
                    epochs=100, batch_size=16, verbose=1, callbacks=[early_stopping])

# Make predictions on the test set
y_pred = model.predict(X_test_scaled)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Evaluate the model
print(classification_report(y_test_classes, y_pred_classes))

# Plot training & validation loss and accuracy values
# Plot loss
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')


# Plot accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='lower right')

plt.show()


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3478/3478 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - accuracy: 0.5817 - loss: 0.9375 - val_accuracy: 0.6121 - val_loss: 0.8820
Epoch 2/100
3473/3478 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6195 - loss: 0.8866

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

# Separate the features and the target variable
X = data_df[['UserReputation', 'AnswerTime']]
y = data_df['Score']

# One-hot encode the target variable
y_encoded = to_categorical(y, num_classes=3)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = Sequential()
model.add(Dense(128, input_dim=2, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss=custom_loss, metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test),
                    epochs=100, batch_size=16, verbose=1, callbacks=[early_stopping])

# Make predictions on the test set
y_pred = model.predict(X_test_scaled)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Evaluate the model
print(classification_report(y_test_classes, y_pred_classes))

# Plot training & validation loss and accuracy values
# Plot loss
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')

# Plot accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
3478/3478 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - accuracy: 0.5569 - loss: 1.5388 - val_accuracy: 0.5941 - val_loss: 1.5159
Epoch 2/100
3471/3478 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6064 - loss: 1.4475

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from tensorflow.keras.regularizers import l2

data = data_df

# Check for and replace infinities and NaNs
data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.dropna(inplace=True)

# Check for and replace infinities and NaNs in the new feature
data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.dropna(inplace=True)

# Separate the features and the target variable
X = data[['UserReputation', 'AnswerTime']]
y = data['Score']

# One-hot encode the target variable
y_encoded = to_categorical(y, num_classes=3)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model with regularization and dropout
model = Sequential()
model.add(Dense(128, input_dim=2, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.3))
model.add(Dense(16, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.3))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss=custom_loss, metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test),epochs=100, batch_size=32, verbose=1, callbacks=[early_stopping])

# Make predictions on the test set
y_pred = model.predict(X_test_scaled)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Evaluate the model
print(classification_report(y_test_classes, y_pred_classes))

# Plot training & validation loss and accuracy values
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper right')

plt.show()


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1739/1739 ━━━━━━━━━━━━━━━━━━━━ 21s 9ms/step - accuracy: 0.4655 - loss: 2.9696 - val_accuracy: 0.5451 - val_loss: 2.1785
Epoch 2/100
1739/1739 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.5577 - loss: 2.0602 - val_accuracy: 0.5687 - val_loss: 1.8230
Epoch 3/100
1739/1739 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.5716 - loss: 1.8185 - val_accuracy: 0.5733 - val_loss: 1.7031
Epoch 4/100
1739/1739 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.5733 - loss: 1.7527 - val_accuracy: 0.5713 - val_loss: 1.6679
Epoch 5/100
1739/1739 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.5775 - loss: 1.6658 - val_accuracy: 0.5754 - val_loss: 1.6167
Epoch 6/100
1739/1739 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.5771 - loss: 1.6709 - val_accuracy: 0.5690 - val_loss: 1.6045
Epoch 7/100
1739/1739 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.5797 - loss: 1.6115 - val_accuracy: 0.5711 - val_loss: 1.6090
Epoch 8/100
1739/1739 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.5776 - loss: 1.6198 - val

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

# Separate the features and the target variable
X = data_df[['UserReputation', 'AnswerTime']]
y = data_df['Score']

# One-hot encode the target variable
y_encoded = to_categorical(y, num_classes=3)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = Sequential()
model.add(Dense(128, input_dim=2, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(optimizer='rmsprop', loss=custom_loss, metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test),
                    epochs=100, batch_size=16, verbose=1, callbacks=[early_stopping])

# Make predictions on the test set
y_pred = model.predict(X_test_scaled)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Evaluate the model
print(classification_report(y_test_classes, y_pred_classes))

# Plot training & validation loss and accuracy values
# Plot loss
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')

# Plot accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3478/3478 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - accuracy: 0.5541 - loss: 1.5891 - val_accuracy: 0.6016 - val_loss: 1.5330
Epoch 2/100
3478/3478 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - accuracy: 0.6071 - loss: 1.4600 - val_accuracy: 0.6162 - val_loss: 1.4793
Epoch 3/100
3478/3478 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - accuracy: 0.6113 - loss: 1.4685 - val_accuracy: 0.6105 - val_loss: 1.4787
Epoch 4/100
3478/3478 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.6111 - loss: 1.4667 - val_accuracy: 0.6163 - val_loss: 1.4630
Epoch 5/100
3478/3478 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.6143 - loss: 1.4970 - val_accuracy: 0.6086 - val_loss: 1.5696
Epoch 6/100
3478/3478 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.6074 - loss: 1.5682 - val_accuracy: 0.6047 - val_loss: 1.5044
Epoch 7/100
3478/3478 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.6135 - loss: 1.5230 - val_accuracy: 0.6051 - val_loss: 1.6297
Epoch 8/100
3478/3478 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - accuracy: 0.6093 - loss: 1.5080

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from tensorflow.keras.regularizers import l2

data = data_df

# Feature Engineering: Add new features
data['Reputation_Time_Ratio'] = data['UserReputation'] / data['AnswerTime']
data['AnswerTime_Squared'] = data['AnswerTime'] ** 2

# Separate the features and the target variable
X = data[['UserReputation', 'AnswerTime', 'Reputation_Time_Ratio', 'AnswerTime_Squared']]
y = data['Score']

# One-hot encode the target variable
y_encoded = to_categorical(y, num_classes=3)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model with regularization and dropout
model = Sequential()
model.add(Dense(64, input_dim=4, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(optimizer=RMSprop(learning_rate=0.0001), loss=custom_loss, metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test),
                    epochs=100, batch_size=32, verbose=1, callbacks=[early_stopping])

# Make predictions on the test set
y_pred = model.predict(X_test_scaled)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Evaluate the model
report = classification_report(y_test_classes, y_pred_classes)
print(report)

# Plot training & validation loss and accuracy values
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper right')

plt.show()


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from tensorflow.keras.regularizers import l2

data = data_df


# Separate the features and the target variable
X = data[['UserReputation', 'AnswerTime']]
y = data['Score']

# One-hot encode the target variable
y_encoded = to_categorical(y, num_classes=3)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model with regularization and dropout
model = Sequential()
model.add(Dense(64, input_dim=2, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(optimizer=RMSprop(learning_rate=0.0001), loss=custom_loss, metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test),
                    epochs=100, batch_size=32, verbose=1, callbacks=[early_stopping])

# Make predictions on the test set
y_pred = model.predict(X_test_scaled)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Evaluate the model
report = classification_report(y_test_classes, y_pred_classes)
print(report)

# Plot training & validation loss and accuracy values
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper right')

plt.show()


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from tensorflow.keras.regularizers import l2

data = data_df

# Feature Engineering: Add new features
data['Reputation_Time_Ratio'] = data['UserReputation'] / data['AnswerTime']
data['AnswerTime_Squared'] = data['AnswerTime'] ** 2

# Separate the features and the target variable
X = data[['UserReputation', 'AnswerTime', 'Reputation_Time_Ratio', 'AnswerTime_Squared']]
y = data['Score']

# One-hot encode the target variable
y_encoded = to_categorical(y, num_classes=3)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model with regularization, dropout, and batch normalization
model = Sequential()
model.add(Dense(128, input_dim=4, activation='relu', kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss=custom_loss, metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test),
                    epochs=200, batch_size=64, verbose=1, callbacks=[early_stopping])

# Make predictions on the test set
y_pred = model.predict(X_test_scaled)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Evaluate the model
report = classification_report(y_test_classes, y_pred_classes)
print(report)

# Plot training & validation loss and accuracy values
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper right')

plt.show()

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

# Load the data
X = data_df[['UserReputation', 'AnswerTime']]
y = data_df['Score']

# One-hot encode the target variable
y_encoded = to_categorical(y, num_classes=3)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = Sequential([
    Dense(128, input_dim=2, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

# Compile the model with a custom loss function or categorical crossentropy
model.compile(optimizer=Adam(learning_rate=0.001), loss=custom_loss, metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test),
                    epochs=100, batch_size=32, verbose=1, callbacks=[early_stopping])

# Make predictions on the test set
y_pred = model.predict(X_test_scaled)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Evaluate the model
print(classification_report(y_test_classes, y_pred_classes))

# Plot training & validation loss and accuracy values
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='lower right')

plt.show()


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

# Load and prepare the data
X = data_df[['UserReputation', 'AnswerTime']].values
y = data_df['Score'].values

# One-hot encode the target variable
y_encoded = to_categorical(y, num_classes=3)

# Reshape X for Conv1D input (samples, timesteps, features)
X = X.reshape((X.shape[0], X.shape[1], 1))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Build the neural network model with Conv1D and MaxPooling1D
model = Sequential([
    Conv1D(filters=32, kernel_size=2, activation='relu', padding='same', input_shape=(X_train.shape[1], X_train.shape[2])),
    MaxPooling1D(pool_size=1),
    Conv1D(filters=64, kernel_size=2, activation='relu', padding='same'),
    MaxPooling1D(pool_size=1),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                    epochs=100, batch_size=32, verbose=1, callbacks=[early_stopping])

# Make predictions on the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Evaluate the model
print(classification_report(y_test_classes, y_pred_classes))

# Plot training & validation loss and accuracy values
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='lower right')

plt.show()
